## THE CIRCUIT CONFIGURATION

In [1]:
# check if notebook is in colab
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import os
import json
import ezkl
import pandas as pd

import torch
from torchvision import transforms
from PIL import Image

import logging
import os
import sys
import sys

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(project_root)
from training.defineSNN import MODSiameseBambooNN


# uncomment for more descriptive logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)


# Specify all the files we need
model_path = os.path.join('ezkl-outputs/snn.onnx')
compiled_model_path = os.path.join('ezkl-outputs/snn.ezkl')
pk_path = os.path.join('ezkl-outputs/pk.key')
vk_path = os.path.join('ezkl-outputs/vk.key')
settings_path = os.path.join('ezkl-outputs/settings.json')
srs_path = os.path.join('ezkl-outputs/kzg.srs')
data_path = os.path.join('ezkl-outputs/input.json')
cal_path = os.path.join('ezkl-outputs/cal_data.json')
##
sol_code_path = os.path.join('../contracts/Verifier.sol')
abi_path = os.path.join('ezkl-outputs/Verifier.abi')
proof_path = os.path.join('ezkl-outputs/proof.json')



In [2]:
### Load the image pair as input ready for inference
class SiameseImageLoader:
    def __init__(self, transform=None):
        self.transform = transform

    def load_and_transform_pair(self, image1_path, image2_path):
        img1 = Image.open(image1_path).convert('L')
        img2 = Image.open(image2_path).convert('L')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        img1 = img1.to(device)
        img2 = img2.to(device)

        return img1, img2

transform = transforms.Compose([
    transforms.Resize((10, 10)),
    transforms.ToTensor(),
])

# Example inputs (TrueNegative, TruePositive, FalseNegative)
# (FalseNegative were produced with a growing rate of bamboo higher than what the model was trained on)
siamese_loader = SiameseImageLoader(transform)
image1_path = "TestValid_t1.jpeg"
image2_path = "TestValid_t0.jpeg"
img1, img2 = siamese_loader.load_and_transform_pair(image1_path, image2_path) # Separate tensors for each image
img1 = img1.unsqueeze(0)
img2 = img2.unsqueeze(0)

print(img1.size())

torch.Size([1, 1, 10, 10])


In [4]:
# Load the trained nn model and make inference on test inputs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = MODSiameseBambooNN().to(device)
loaded_model.load_state_dict(torch.load('../training/trained_simplesnn_lr_0.01.pth', map_location=torch.device('cpu')))
loaded_model.eval()

with torch.no_grad():
    finalout = loaded_model(img1, img2)
    predicted_label = (torch.sigmoid(finalout) > 0.5).item()

print(f'Same class probability: {torch.sigmoid(finalout)}')
print(f'Predicted Label: {predicted_label}')

# Calculate the total number of parameters
total_params = sum(p.numel() for p in loaded_model.parameters())
print(f'Total Parameters: {total_params}')

Same class probability: tensor([[0.6949]])
Predicted Label: True
Total Parameters: 30849


In [5]:
# Export the loaded Siamese Network model to ONNX
torch.onnx.export(loaded_model,                      # loaded Siamese Network model
                  (img1, img2),                      # model input (or a tuple for multiple inputs)
                  model_path,                        # where to save the ONNX model
                  export_params=True,                # store the trained parameter weights inside the model file
                  opset_version=14,                  # the ONNX version to export the model to
                  do_constant_folding=True,          # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})



# Move tensors from GPU to CPU
img1_cpu = img1.cpu()
img2_cpu = img2.cpu()
print(img1_cpu.size())

# Serialize data into a JSON file
input_data = [
    img1_cpu.detach().numpy().reshape([-1]).tolist(),
    img2_cpu.detach().numpy().reshape([-1]).tolist(),
]

data = dict(input_data=input_data)
json.dump(data, open(data_path, 'w'))

torch.Size([1, 1, 10, 10])


In [6]:
# *** Providing the calibrate settings function with a larger and broader range of sample inputs.


csv_path = '../training/dataset/forest_dataset.csv'
df = pd.read_csv(csv_path)

# Construct the full paths to the images
img1_files = [os.path.join('..', 'training', path) for path in df['imageT0'].iloc[:20]]
img2_files = [os.path.join('..', 'training', path) for path in df['imageT1'].iloc[:20]]

img1_list = []
img2_list = []
for img1_path, img2_path in zip(img1_files, img2_files):
    img1, img2 = siamese_loader.load_and_transform_pair(img1_path, img2_path)
    img1 = img1.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img2 = img2.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img1_list.extend(img1) # don't use .append bc it should look like [[xxx...],[yyy...]]
    img2_list.extend(img2)

data = dict(input_data = [img1_list, img2_list])
# Serialize data into file:
json.dump( data, open(cal_path, 'w' ))

In [7]:
#For ezkl to compute a snark, it needs some settings to determine how to create the circuit.
#This cell instantiates some parameters that determine the circuit shape, size etc
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs
py_run_args.variables = [("batch_size", 1)]

!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True
print("gen_settings OK")

# *** SCALES is one of main knobs you can turn to trade off accuracy for proving efficiency
# Under the hood calibration iterates over the scales array to see how precise you can go before failure. 
# For example if 7 fails it falls back to 1.
# (Right now the default scales values for the resources target is 8-10 and 10-13  for accuracy)
res = ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources", scales = [1, 7])
assert res == True

INFO ezkl.graph.model 2024-01-05 15:42:20,173 model.rs:775 set batch_size to 1
INFO ezkl.graph.model 2024-01-05 15:42:20,861 model.rs:477 model has 1 instances
INFO ezkl.graph.model 2024-01-05 15:42:20,882 model.rs:1398 calculating num of constraints using dummy model layout...
INFO ezkl.graph.model 2024-01-05 15:42:21,990 model.rs:1473 model uses 57011 rows (coord=114023, constants=97549)
INFO ezkl.graph.model 2024-01-05 15:42:22,110 model.rs:775 set batch_size to 1


gen_settings OK


INFO ezkl.execute 2024-01-05 15:42:22,473 execute.rs:635 num of calibration batches: 20
INFO ezkl.graph.model 2024-01-05 15:42:22,599 model.rs:775 set batch_size to 1
INFO ezkl.graph.model 2024-01-05 15:42:23,146 model.rs:477 model has 1 instances
INFO ezkl.graph.model 2024-01-05 15:42:23,150 model.rs:1398 calculating num of constraints using dummy model layout...
INFO ezkl.graph.model 2024-01-05 15:42:24,164 model.rs:1473 model uses 25736 rows (coord=51473, constants=34929)
INFO ezkl.graph 2024-01-05 15:42:24,167 mod.rs:748 input scales: [1, 1]
INFO ezkl.graph.model 2024-01-05 15:42:25,339 model.rs:477 model has 1 instances
INFO ezkl.graph.model 2024-01-05 15:42:25,341 model.rs:1398 calculating num of constraints using dummy model layout...
INFO ezkl.graph.model 2024-01-05 15:42:25,950 model.rs:1473 model uses 25736 rows (coord=51473, constants=34929)
INFO ezkl.graph 2024-01-05 15:42:25,962 mod.rs:1025 setting lookup_range to: (-74, 82), setting logrows to: 16
INFO ezkl.graph.model 20

In [8]:
# Now compile the model into a circuit
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

INFO ezkl.graph.model 2024-01-05 15:48:54,562 model.rs:775 set batch_size to 1


In [13]:
# get public srs from kzg ceremony, saved to srs path.
res = ezkl.get_srs(settings_path)
assert res == True

INFO ezkl.execute 2024-01-05 15:53:01,790 execute.rs:451 SRS does not exist, downloading...


INFO ezkl.execute 2024-01-05 15:53:03,365 execute.rs:467 SRS downloaded


In [15]:
# Setup the circuit and make sure the keys are generated afterwards.
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

INFO ezkl.pfsys.srs 2024-01-05 15:57:06,085 srs.rs:23 loading srs from "/home/manu/.ezkl/srs/kzg16.srs"
INFO ezkl.execute 2024-01-05 15:57:06,098 execute.rs:1782 downsizing params to 16 logrows
INFO ezkl.graph.vars 2024-01-05 15:57:06,158 vars.rs:422 number of blinding factors: 5
INFO ezkl.graph.model 2024-01-05 15:57:06,173 model.rs:1076 configuring model
INFO ezkl.graph 2024-01-05 15:57:06,176 mod.rs:1361 circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
WARNING ezkl.circuit.modules.planner 2024-01-05 15:57:06,201 planner.rs:122 spawning module 2
INFO ezkl.graph.model 2024-01-05 15:57:06,294 model.rs:1110 model layout...


INFO ezkl.graph.model 2024-01-05 15:57:10,313 model.rs:1201 model uses 24636 rows (coord=49272, constants=31500)
INFO ezkl.pfsys 2024-01-05 15:57:40,375 mod.rs:401 VK took 34.217
INFO ezkl.graph.vars 2024-01-05 15:57:40,378 vars.rs:422 number of blinding factors: 5
INFO ezkl.graph.model 2024-01-05 15:57:40,379 model.rs:1076 configuring model
INFO ezkl.graph 2024-01-05 15:57:40,381 mod.rs:1361 circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
WARNING ezkl.circuit.modules.planner 2024-01-05 15:57:40,506 planner.rs:122 spawning module 2
INFO ezkl.graph.model 2024-01-05 15:57:40,508 model.rs:1110 model layout...
INFO ezkl.graph.model 2024-01-05 15:57:43,608 model.rs:1201 model uses 24636 rows (coord=49272, constants=31500)
INFO ezkl.pfsys 2024-01-05 15:57:56,782 mod.rs:407 PK took 16.393
INFO ezkl.pfsys 2024-01-05 15:57:56,784 mod.rs:674 saving verification key 💾
INFO ezkl.pfsys 2024-01-05 15:57:56,852 mod.rs

In [16]:
# GENERATE the verifier associated with the circuit
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "solc-select"])
    !solc-select install 0.8.20
    !solc-select use 0.8.20
    !solc --version
# rely on local installation if the notebook is not in colab
except:
    pass


In [17]:
res = ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path
    )
assert res == True
assert os.path.isfile(sol_code_path)

INFO ezkl.execute 2024-01-05 15:59:02,806 execute.rs:73 checking solc installation..
INFO ezkl.pfsys.srs 2024-01-05 15:59:02,863 srs.rs:23 loading srs from "/home/manu/.ezkl/srs/kzg16.srs"
INFO ezkl.execute 2024-01-05 15:59:02,948 execute.rs:1782 downsizing params to 16 logrows
INFO ezkl.pfsys 2024-01-05 15:59:02,992 mod.rs:614 loading verification key from "ezkl-outputs/vk.key"
INFO ezkl.graph.vars 2024-01-05 15:59:02,999 vars.rs:422 number of blinding factors: 5
INFO ezkl.graph.model 2024-01-05 15:59:03,001 model.rs:1076 configuring model
INFO ezkl.graph 2024-01-05 15:59:03,009 mod.rs:1361 circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}


## PROVE and VERIFY
### here we will generate and verify a proof locally. Then we will format the inputs and the proof in a way compatible for the evm verifier

In [18]:
witness_path = os.path.join('ezkl-outputs/witness.json')
# generate the witness file
res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

INFO ezkl.graph 2024-01-05 15:59:53,101 mod.rs:767 input scales: [1, 1]


In [20]:
# !!!Generate the proof!!!

proof = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

print(proof)
assert os.path.isfile(proof_path)

INFO ezkl.pfsys.srs 2024-01-05 16:00:27,703 srs.rs:23 loading srs from "/home/manu/.ezkl/srs/kzg16.srs"
INFO ezkl.execute 2024-01-05 16:00:27,797 execute.rs:1782 downsizing params to 16 logrows
INFO ezkl.pfsys 2024-01-05 16:00:27,805 mod.rs:636 loading proving key from "ezkl-outputs/pk.key"
INFO ezkl.graph.vars 2024-01-05 16:00:27,805 vars.rs:422 number of blinding factors: 5
INFO ezkl.graph.model 2024-01-05 16:00:27,811 model.rs:1076 configuring model
INFO ezkl.graph 2024-01-05 16:00:27,836 mod.rs:1361 circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO ezkl.pfsys 2024-01-05 16:00:32,732 mod.rs:470 proof started...
INFO ezkl.graph.vars 2024-01-05 16:00:32,735 vars.rs:422 number of blinding factors: 5
INFO ezkl.graph.model 2024-01-05 16:00:32,746 model.rs:1076 configuring model
INFO ezkl.graph 2024-01-05 16:00:32,748 mod.rs:1361 circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num

{'instances': [[[11874046804959952378, 12031363491236821520, 978108733370872704, 1515817019597253525]]], 'proof': '2b74945ff68f0943e86b6f7c27c5ae38d6db4a38021f489203ccf7bb14d209551eec409a83f879f98a247a2b834c1e2671f7ca425d611b3e7abf6415f17ff4d7093b324fc7637a95453cd4500b753b4dde9e35256f8cac4e2913714d0a94e22229125581e945f01e88e6148cbac213c11069128fb2515344c279c1e7d3dbf45926c4071e8a69115307aa30f7be31fd53ac560a7de8b7004205e750b923f627a220f654f4ad70b635cb85ea811aa7a894e9b8e1fb4f942e9a54c7e03a601009170a68bcd80296f6bd506da2be667e12480b0eef47b134c489fe0e45ce9cccae932658ebd4edf582e935b91a5c1be3397cd4779af8fbaad20f8dd612674a3df7340e65c78eb63851020999ca77023ad47a4551da6a1eaa6d942b227ccb573ac3c4130abdb92367fb9cf044027acda4e23f9d50dd14e31ccac5ebaa1474334b312207d2dcfafaad9a9ed501bdf0801de722c07aec36649cbd912effc8e327ab55d501426d64615cafa2d68dfed48884348377d9e5f9865e379abcc76a95cb9e5b0b044860f1ea601e72b09c0c08a1392cc77282a0c5a8bd4acbe25a47f1ea0f6b9129ff321cbdb065bd30f0ebc0ab0aa954f60a64faffcdc8f53feb9

In [21]:
# Sanity check off-chain verification
res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

assert res == True
print("verified locally")

INFO ezkl.pfsys.srs 2024-01-05 16:02:14,369 srs.rs:23 loading srs from "/home/manu/.ezkl/srs/kzg16.srs"
INFO ezkl.execute 2024-01-05 16:02:14,459 execute.rs:1782 downsizing params to 16 logrows
INFO ezkl.pfsys 2024-01-05 16:02:14,471 mod.rs:614 loading verification key from "ezkl-outputs/vk.key"
INFO ezkl.graph.vars 2024-01-05 16:02:14,478 vars.rs:422 number of blinding factors: 5
INFO ezkl.graph.model 2024-01-05 16:02:14,482 model.rs:1076 configuring model
INFO ezkl.graph 2024-01-05 16:02:14,486 mod.rs:1361 circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO ezkl.execute 2024-01-05 16:02:14,621 execute.rs:1740 verify took 0.100
INFO ezkl.execute 2024-01-05 16:02:14,624 execute.rs:1745 verified: true


verified locally


In [22]:
# Get input data for the verifier contract (evmInputs.json)
onchain_input_array = []
# avoiding printing last comma
formatted_output = "["
for i, value in enumerate(proof["instances"]):
    for j, field_element in enumerate(value):
        onchain_input_array.append(ezkl.vecu64_to_felt(field_element))
        formatted_output += str(onchain_input_array[-1])
        if j != len(value) - 1:
            formatted_output += ", "
    formatted_output += "]"
print("form output",formatted_output )
    
input_list = [int(entry, 0) for entry in formatted_output.strip('[]').split(', ')]
quoted_list = [format(entry, "#066x") for entry in input_list]
json_data = {
    "instances": quoted_list,
    "proof": "0x" + proof["proof"]
}
with open('evmInputs.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)
print("Saved evmInputs.json: ", json_data)


form output [0x0000000000000000000000000000000000000000000000000000000000000062]
Saved evmInputs.json:  {'instances': ['0x0000000000000000000000000000000000000000000000000000000000000062'], 'proof': '0x2b74945ff68f0943e86b6f7c27c5ae38d6db4a38021f489203ccf7bb14d209551eec409a83f879f98a247a2b834c1e2671f7ca425d611b3e7abf6415f17ff4d7093b324fc7637a95453cd4500b753b4dde9e35256f8cac4e2913714d0a94e22229125581e945f01e88e6148cbac213c11069128fb2515344c279c1e7d3dbf45926c4071e8a69115307aa30f7be31fd53ac560a7de8b7004205e750b923f627a220f654f4ad70b635cb85ea811aa7a894e9b8e1fb4f942e9a54c7e03a601009170a68bcd80296f6bd506da2be667e12480b0eef47b134c489fe0e45ce9cccae932658ebd4edf582e935b91a5c1be3397cd4779af8fbaad20f8dd612674a3df7340e65c78eb63851020999ca77023ad47a4551da6a1eaa6d942b227ccb573ac3c4130abdb92367fb9cf044027acda4e23f9d50dd14e31ccac5ebaa1474334b312207d2dcfafaad9a9ed501bdf0801de722c07aec36649cbd912effc8e327ab55d501426d64615cafa2d68dfed48884348377d9e5f9865e379abcc76a95cb9e5b0b044860f1ea601e72b09c0c08a1392c